In [1]:
import pandas as pd
import datetime
import xlsxwriter

try:
    df = pd.read_excel(open("rptRackReport_13_14.xls",'rb'), header = 1)
except:
    df = pd.read_excel(open("rptRackReport.xls",'rb'), header = 1)
    
data = df[:-1]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [2]:
#Excel uses UTC epoch (1900-01-00); python uses Unix epoch (1970-01-01)
#create function to convert Unix to UTC
def convert_Unix_to_UTC(time):
    epoch_Unix = datetime.datetime(1970,1,1)
    epoch_UTC = datetime.datetime(1899,12,31)
    diff = epoch_Unix - epoch_UTC
    return (time - diff - datetime.timedelta(days=1)) #subtract 1 day to account for 'inclusive' time difference

data["Text152"] = pd.to_datetime(data["Text152"],unit='d').apply(convert_Unix_to_UTC)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
#create year-month column
data["year_month"] = data["Text152"].dt.strftime('%Y-%m')

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [4]:
#C1.25 calcs

#sum gallons loaded per month
thruput = data.groupby(['year_month'])['Text125'].sum()

#compliance status
status_thru = []
for thru in thruput:
    if thru <= 6083333:
        stat = 'OK'
    else:
        stat = 'EXCEEDANCE'
    status_thru.append(stat)

In [5]:
#C6.5 calcs

#find max temps for each month
max_temps = data.groupby(['year_month'])['Text122'].max()

#compliance status
status_temp = []
for temp in max_temps:
    if temp <= 500:
        stat = 'OK'
    else:
        stat = 'EXCEEDANCE'
    status_temp.append(stat)

In [6]:
#extract YYYY-MM dates to write to Excel
year_months = pd.unique(data["year_month"])

In [8]:
#write to Excel
#setup workbook and worksheet
workbook = xlsxwriter.Workbook('%s to %s_Racks 13 & 14_C1.11 & C6.5.xlsx' %(year_months[0],year_months[-1]))
bold = workbook.add_format({'bold':True})
num_fmt = workbook.add_format({'num_format':'###,###,###'})
worksheet1 = workbook.add_worksheet('Summary')

worksheet1.set_column('B:D',15)

row = 0
col = 0

#write C1.11 headerss
worksheet1.write(row,col,"C1.11: Racks 13 and 14 (D29) cannot exceed monthly throughput of 6,083,333 gallons.",bold)
row += 2
columns = ['Date','Throughput (gal)','Limit (gal)','Compliance']
for header in columns:
    worksheet1.write(row,col,header)
    col += 1
row += 1
col = 0

#write C1.11 data
for i in range(len(thruput)):
    worksheet1.write(row,col,year_months[i])
    col += 1
    worksheet1.write(row,col,thruput[i],num_fmt)
    col += 1
    worksheet1.write_number(row,col,6083333,num_fmt)
    col += 1
    worksheet1.write(row,col,status_thru[i])
    row += 1
    col = 0
    
row += 1    
#write C6.5 headers
worksheet1.write(row,col,"C6.5: Racks 13 and 14 (D29) cannot exceed temperature of 500F.",bold)
row += 2
columns = ['Date','Max Temp (F)','Limit (F)','Compliance']
for header in columns:
    worksheet1.write(row,col,header)
    col += 1
row += 1
col = 0

#write C6.5 data
for i in range(len(thruput)):
    worksheet1.write(row,col,year_months[i])
    col += 1
    worksheet1.write(row,col,max_temps[i],num_fmt)
    col += 1
    worksheet1.write_number(row,col,500,num_fmt)
    col += 1
    worksheet1.write(row,col,status_temp[i])
    row += 1
    col = 0
    
workbook.close()